# North Sea Wind Farms

## Goal

The goal of this notebook is to collect (spatial) data and generate statistics on wind farms in the North Sea. We want information on:
1. Spatial status and changes
2. Temporal changes
3. Technical specifics on turbines (power, infrastructure incl. cabels, materials, supply chain)
5. Ownership
6. End-users

## Data sources

### Location and ownership
1. Global Offshore Wind Farm Database And Intelligence, 4COffshore ([source](https://www.4coffshore.com/windfarms/)). Holds information on location, ownership and production of wind farms. Is a commercial provider. Some data seem a bit messy (e.g. the categorisation into sea areas is not consistent. 
2. Wind Europe Public database ([source](https://windeurope.org/intelligence-platform/product/european-offshore-wind-farms-map-public/)). Holds information on location and ownership. Is a lobby organisation. Dataset is probably not complete. 
3. EMODnet Wind Turbines ([source](https://emodnet.ec.europa.eu/geonetwork/emodnet/eng/catalog.search#/search?any=EMODnet%20Human%20Activities,%20Wind%20Farms)). Norway data is lacking. Data is current and has been extensively verified up until January 1st this year, so this is a very useful resource.
4. Crown Estate offshore wind farm onwership tables ([source](https://www.thecrownestate.co.uk/en-gb/what-we-do/on-the-seabed/energy/offshore-wind-farm-ownership/#OFTOownership)). More data available at the Crown Estate open data portal ([source](https://opendata-thecrownestate.opendata.arcgis.com/search?groupIds=f0d0ec92da76434d9e91f2e4dcb3a99f)). And here for Scotland ([source](https://crown-estate-scotland-spatial-hub-coregis.hub.arcgis.com/search?tags=offshore%20wind)). Official UK registries. Data quality is presumed to be good.
5. NVE data from Norway ([source](https://nedlasting.nve.no/gis/)). Offical Norwegian registry. Data quality is presumed to be good.
6. Netherlands wind turbines from RIVM ([source](https://data.rivm.nl/meta/srv/dut/catalog.search#/search?resultType=details&sortBy=relevance&fast=index&_content_type=json&from=1&to=20&any=Windturbines%20-%20vermogen)). Data quality is good. Is point data instead of polygons. Info on individual turbines (height, power, location).
7. Denmark
8. Germany
9. Belgium

### Tenders and supply chain
1. Bloomberg Terminal (at office of FTM): contains information on the supply chain of wind turbine manufacturers. Some data might be outdated. 
2. Aleph (FTM data repository) contains all European Tenders

In [ ]:
import pandas as pd
import geopandas as gpd
import ast

In [ ]:
PATH_DATA = '../../data/renewables/'

#### 4COffshore

The latitude is given, but the longitude is not. We therefore might need to link the data on project names

In [ ]:
# Import 4COffshore

_4c = pd.read_csv(f'{PATH_DATA}4c_offshore_freemium_windfarms.csv')

# Filter out North Sea Countries

_4c = _4c[_4c.country_filter.isin(['netherlands', 'norway', 'united-kingdom', 'germany', 'denmark', 'belgium'])].copy()

# Rename columns

new_cols = {'country_filter': 'country',
            'owners': 'owner',
            'windfarm_capacity': 'power_mw',
            'development_status': 'status'}

_4c = _4c.rename(columns=new_cols)

# Clean latitude column

_4c.latitude = _4c.latitude.str.replace('°', '', regex=False)

# Clean power_mw column

_4c.power_mw = _4c.power_mw.str.replace(' MW', '').astype(float)
            
# Select relevant columns

_4c = _4c[['name', 'other_names', 'country', 'owner', 'developers', 'status', 'power_mw', 'category_round', 'main_url', 'comments', 'turbine_model', 'foundation_type', 'latitude']].copy()

# Clean owners to proper list

_4c[['developers', 'owner']] = _4c[['developers', 'owner']].applymap(lambda x: ast.literal_eval(x))

# Get list of project names for comparison

_4c_names = list(set(_4c.name.str.lower()))
len(_4c)

#### Wind Europe

The dataset contains coordinates, so we can perform a spatial join to link the data to the polygons of EMODNet. But let's also get the project names to see if we can match some of the data from 4COffshore. There might be some wind farms outside of the North Sea, but we can use a spatial mask to filter these out.

In [ ]:
# Import Wind Europe

weurope = pd.read_csv(f'{PATH_DATA}windeurope.csv', sep=';')

# Clean columns

weurope.columns = weurope.columns.str.lower().str.replace(' ', '_', regex=False)

# Drop some columns

weurope = weurope.drop([col for col in weurope.columns if '(group)' in col], axis=1)

# Rename columns

new_cols = {'project_name': 'name',
            'total_project_capacity': 'power_mw',
            'to_show_turbine_manufacturer': 'turbine_manufacturer',
            'commissioning_date': 'year',
            'gis_latitude': 'latitude',
            'gis_longitude': 'longitude',
            'number_of_turbines': 'n_turbines'}

weurope = weurope.rename(columns=new_cols)

# Select relevant columns

weurope = weurope[['name', 'status', 'power_mw', 'year', 'year_awarded', 'foundation_type', 'owner', 'turbine_manufacturer', 'latitude', 'longitude']].copy()

# Get list of project names for comparison

weurope_names = list(set(weurope['name'].str.lower()))
len(weurope_names)

#### EMODnet

This data is the basis of our efforts, because it's very current and reliable. 

In [ ]:
# Import EMODnet

emod = gpd.read_file(f'{PATH_DATA}EMODnet_HA_WindFarms_20221219/EMODnet_HA_WindFarms_pg_20221219.shp')# Filter out North Sea countries

# Filter North Sea countries

emod = emod[emod.COUNTRY.isin(['Denmark', 'United Kingdom', 'Netherlands', 'Germany', 'Belgium'])].copy()

# Column names to lowercase

emod.columns = emod.columns.str.lower()

# Select relevant columns

emod = emod[['country', 'name', 'n_turbines', 'power_mw', 'status', 'year', 'geometry']].copy().reset_index().drop('index', axis=1)

# Get list of project names for comparison

emod_names = list(set(emod.name.str.lower()))
len(emod_names)

#### NVE Norwegian data

There are two slightly different datasets available:
1. From the NVE map interface (norwegian_wind_farms.geojson). This is the most elaborate one and also contains ownership information. 
2. Also from the NVE but from the open data service. This has fewer data fields, but contain 4 more records.

In [ ]:
nve = gpd.read_file(f'{PATH_DATA}norwegian_wind_farms.geojson')
nve2 = gpd.read_file(f'{PATH_DATA}NVE_60751B14_1686171454912_10368/NVEData/Vindkraft_VindkraftanleggOmr.geojson')